**Rationale** This assignment will help you practice running and interpreting basic regressions analysis.

**[Datasets](https://drive.google.com/drive/folders/1LJ38ctVX62ggGGygJDLlLG2V6jZ4BNml?usp=sharing)**
1. [Safegraph Temple Visitor Sample](https://drive.google.com/file/d/1u_sDxFp9fja-_cop4yzIAJ_hX2-fplWN/view?usp=sharing) 
1. [Avocado Prices](https://drive.google.com/file/d/1QJ_dyyBFkHhC6VkzVvdAFa385mtqtntM/view?usp=sharing)

**Data description** This dataset includes SafeGraph's sampled foot traffic volume for Temple University between 1-1-2019 and 5-31-2020. Safegraph collects the foot traffic data from a network of phone apps. You can view the `visits` data as the number of SafeGraph tracked devices to visit Temple University. This dataset is merged with Philadelphia's weather data. 


# 1. Run and interpret a simple linear regression (5 points)...
...to figure out the relationship between daily average temperateure as measured in degrees fahrenheit, `"temp"`, and people on campus `"visits"` for the Fall 2019 semester (limit data to M-F 8-26 - 11-24-2019, inclusive).

In your answer: 
1. First, look at data. What is the average number of SafeGraph visitors to Temple University during the teaching days of the Fall 2019 semester? Roughly what percent of the Temple student population is this?
1. Specify (write out the regression equation), run, and interpret a regression that documents the relationship between temperature and foot traffic (visits) on campus.
    1. Do more students visit campus (and presumably go to class) when the weather is warmer or colder?
    1. Besides students not wanting to go to class when the temperature is not to their liking, what else might cause the observed relationship between visits and temperature?

In [1]:
# imports here
import os, pandas as pd, matplotlib.pyplot as plt
from google.colab import drive
from statsmodels.formula import api as smf
drive.mount('drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at drive


In [2]:
# list directory containing the data
fpath = 'drive/MyDrive/A6/' # change this to your data folder
os.listdir(fpath)

['temple_traffic.csv', 'avocado.csv', 'retail_data']

In [3]:
# read in the dataset
temple = pd.read_csv(fpath + 'temple_traffic.csv', index_col = 0)

In [4]:
temple

,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,count_slp,stp,count_stp,visib,count_visib,wdsp,count_wdsp,mxspd,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt
0,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-02,65,997286.0,99999.0,45.8,24.0,NaN,0.0,1016.8,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,63.3,*,34.7,*,0.0,I,NaN,0.0
1,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-03,68,997286.0,99999.0,53.0,24.0,NaN,0.0,1006.3,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,59.9,*,45.9,*,0.0,I,NaN,0.0
2,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-04,71,997286.0,99999.0,51.4,24.0,NaN,0.0,1006.3,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,58.6,*,45.1,*,0.0,I,NaN,0.0
3,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-05,70,997286.0,99999.0,49.2,24.0,NaN,0.0,1017.0,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,55.8,*,41.7,*,0.0,I,NaN,0.0
4,sg:00579e722c8e48178ef0c66a7c91f92c,2020-03-06,57,997286.0,99999.0,41.7,24.0,NaN,0.0,1013.1,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,49.1,*,37.2,*,0.0,I,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,sg:00579e722c8e48178ef0c66a7c91f92c,2020-02-19,122,997286.0,99999.0,49.4,24.0,NaN,0.0,1020.9,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,58.1,*,43.0,*,0.0,I,NaN,0.0
514,sg:00579e722c8e48178ef0c66a7c91f92c,2020-02-20,100,997286.0,99999.0,37.1,24.0,NaN,0.0,1030.0,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,45.3,*,31.6,*,0.0,I,NaN,0.0
515,sg:00579e722c8e48178ef0c66a7c91f92c,2020-02-21,107,997286.0,99999.0,29.7,24.0,NaN,0.0,1031.6,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,36.1,*,23.0,*,0.0,I,NaN,0.0
516,sg:00579e722c8e48178ef0c66a7c91f92c,2020-02-22,63,997286.0,99999.0,37.1,24.0,NaN,0.0,1026.9,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,51.3,*,28.4,*,0.0,I,NaN,0.0


In [5]:
# convert the date column to pandas datetime using pd.to_datetime(...)
# if you've fogotten, look through notes from module 3
temple.date= pd.to_datetime(temple.date)


In [6]:
# create a new column called 'dow' (for day of week) using temple.date.dt.weekday
# note that Monday through Sunday corresponds to 0 through 6

temple['dow'] = temple.date.dt.weekday



In [7]:
# select only the rows where the date lies between 2019/8/26 and 2019/11/24 (inclusive) 
# and correspond to a regular school date (e.g. are M-F)
# call the reuslting dataframe F2019
# remember you can select rows using df.loc[conditions]
# If you forgot how to do this, look at the notes and in class exercise from module 3
F2019 = temple.loc[temple.date >= '2019/8/26'] 
F2019 = temple.loc[temple.date <= '2019/11/24']

cond = (temple.date > '2019/8/26') & (temple.date < '2019/11/24') & (temple.dow <=4)
F2019 = temple.loc[cond]

# must add M-F
#dow does not show up on cond

In [8]:
F2019

,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,count_slp,stp,count_stp,visib,count_visib,wdsp,count_wdsp,mxspd,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt,dow
7,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-23,169,997286.0,99999.0,80.2,23.0,NaN,0.0,1012.9,23.0,NaN,0.0,NaN,0.0,0.0,23.0,NaN,NaN,90.7,*,70.9,*,0.0,I,NaN,0.0,0
8,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-24,182,997286.0,99999.0,75.3,24.0,NaN,0.0,1009.2,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,83.3,*,66.7,*,0.0,I,NaN,0.0,1
9,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-25,156,997286.0,99999.0,72.3,24.0,NaN,0.0,1011.7,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,81.0,*,61.7,*,0.0,I,NaN,0.0,2
10,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-26,168,997286.0,99999.0,72.1,24.0,NaN,0.0,1009.5,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,87.3,*,63.5,*,0.0,I,NaN,0.0,3
11,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-27,165,997286.0,99999.0,70.3,22.0,NaN,0.0,1017.1,22.0,NaN,0.0,NaN,0.0,0.0,22.0,NaN,NaN,79.3,*,63.7,*,0.0,I,NaN,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,sg:00579e722c8e48178ef0c66a7c91f92c,2019-11-11,117,997286.0,99999.0,51.7,23.0,NaN,0.0,1016.7,23.0,NaN,0.0,NaN,0.0,0.0,23.0,NaN,NaN,63.5,*,42.4,*,0.0,I,NaN,0.0,0
505,sg:00579e722c8e48178ef0c66a7c91f92c,2019-11-12,106,997286.0,99999.0,47.8,24.0,NaN,0.0,1010.9,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,57.9,*,34.5,*,0.0,I,NaN,0.0,1
506,sg:00579e722c8e48178ef0c66a7c91f92c,2019-11-13,106,997286.0,99999.0,30.5,23.0,NaN,0.0,1025.9,23.0,NaN,0.0,NaN,0.0,0.0,23.0,NaN,NaN,35.6,*,24.8,*,0.0,I,NaN,0.0,2
507,sg:00579e722c8e48178ef0c66a7c91f92c,2019-11-14,107,997286.0,99999.0,35.6,22.0,NaN,0.0,1029.5,22.0,NaN,0.0,NaN,0.0,0.0,22.0,NaN,NaN,45.9,*,28.9,*,0.0,I,NaN,0.0,3


In [9]:
# find the maximum date of F2019, make sure that it is less than or equal to 2019/11/24
F2019['date'].max()


Timestamp('2019-11-22 00:00:00')

In [10]:
# find the minimum date of F2019, make sure that it is greater than or equal to 2019/8/26
F2019['date'].min()


Timestamp('2019-08-27 00:00:00')

In [11]:
# display the unique values of dow in F2019 (use the .unique() method on dow column)
# make sure there are only M-F (e.g. 0-4)

F2019['dow'].unique()

array([0, 1, 2, 3, 4])

In [12]:
# compute the mean visits

F2019['visits'].mean()

137.296875

In [13]:
# look up the total number of temple students in 2019 and compute the percentage the above average represents

print(137.296875/ 38794)


0.003539126540186627


**Edit this cell**
The average number of visitors in the sample is **__137.296875__**, this is approximately **_.00354_**\% of the Temple student population.

**Edit this cell**

Before running a regression to document the relationship between temperature and the sampled visitors, answer the following.

The logical dependent variable should be **__visits__**, the logical indepdent (aka explanatory) variable should be **____temperature___** because **____temperature____** should cause changes in **__visits__** and not the other way around.


Consequently, the following regression equation describes the relationship between temperature and foot traffic (change the $x$ and $y$ to the approriate variable names):

$$
y = \beta_0 +\beta_1x + e 
$$


In [14]:
# run the above regression using statsmodels
# call the resulting regression results object the variable "res"

formula = smf.ols('visits ~ temperature', data=)


SyntaxError: ignored

In [15]:
# print the regression summary table (e.g. print(res.summary()))
print(res.summary())


NameError: ignored

**Edit this cell**

The regression's R$^2$ is **___.002___**, this suggests that **___.2__**\% of the variation in **___visits_____** is explained by **___temperature____**.

The intercept ($\beta_0$) is **___87.0039__**, this means that when **__temperature___** is 0 **degrees fahreinheit**, **___visits___** is on average **____87.0039__**. This is statistically **significant(choose one)**, meaning that it is different from **___temperature___** in the population.

The coefficient ($\beta_1$) for **___temperature___** is **____0.1281__**. This means that for every **___one___** increase in **____visitors_**, **___temperature_** **decreases (choose one)** by **___0.1281___**. This is statistically **significant(choose one)**, meaning that it is different from **____temperature___** in the population. In other words, we can reject the null hypothesis that there is no effect of temperature on campus foot traffic.

# 2. Avocado price and demand (5 points) (multiple regression)

For this exercise, use our familiar avocado dataset.

For the following cities, run a multiple regression that investigates the relationship between demand and price while simultaneously accounting for the effect or regions and avocado types.

Use only the conventional avocado data from the following cities:

```
cities = ['Philadelphia', 'NewYork', 'BaltimoreWashington', 'Boston', 'Chicago', 'SanFrancisco']
```


In [17]:
# read in the dataset, call it avocado
avocado = pd.read_csv(fpath + 'avocado.csv', index_col = 0)
avocado.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [18]:
# select only the rows corresponding to the cities listed above
# call the resulting dataframe avocado (replace avocado with the result of this subset selection)
# use .isin(...) method (see notes from module 4 bar plot to find an example of this)
# also make sure to select only the data for the conventional avocado type
cities = ['Philadelphia', ' NewYork', 'BaltimoreWashington', 'Boston', 'Chicago', 'San Francisco']
avocado = avocado[(avocado.region.isin(cities))&(avocado.type=='conventional')].copy()


In [19]:
avocado

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
104,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.00,conventional,2015,BaltimoreWashington
105,2015-12-20,1.11,642682.40,44741.37,434590.82,19112.14,144238.07,141751.91,2486.16,0.00,conventional,2015,BaltimoreWashington
106,2015-12-13,1.15,619509.33,44400.26,399851.59,24291.20,150966.28,149070.40,1895.88,0.00,conventional,2015,BaltimoreWashington
107,2015-12-06,1.04,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.00,conventional,2015,BaltimoreWashington
108,2015-11-29,1.16,545800.05,41028.15,379156.09,20147.53,105468.28,103615.95,1852.33,0.00,conventional,2015,BaltimoreWashington
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8869,2018-02-04,1.12,819224.30,62044.22,467927.80,645.87,288606.41,254149.59,33540.82,916.00,conventional,2018,Philadelphia
8870,2018-01-28,1.23,537883.88,90381.50,256525.12,390.07,190587.19,183977.34,6289.85,320.00,conventional,2018,Philadelphia
8871,2018-01-21,1.32,527860.09,69862.88,303263.33,383.99,154349.89,144550.12,9797.01,2.76,conventional,2018,Philadelphia
8872,2018-01-14,1.50,419189.36,36351.30,232190.19,443.69,150204.18,126685.84,23512.81,5.53,conventional,2018,Philadelphia


In [20]:
# show the unique regions in the resulting dataframe 
# make sure we have only the ones we wanted to select from above

avocado.region.unique()



array(['BaltimoreWashington', 'Boston', 'Chicago', 'Philadelphia'],
      dtype=object)

In [21]:
# show the unique types in the resulting dataframe 
# make sure we have only conventional

avocado.type.unique()

array(['conventional'], dtype=object)

**edit this cell**
To model the demand of avocaldos, we want to run the following regression:

$$
Demand = \beta_0 + \beta_1 Price +\beta_2 Boston +\beta_3Chicago +\beta_4NewYork + \beta_5 Philadelphia + \beta_6 SanFrancisco+e
$$

In this equation (given the cities we've selected for the dataset), $\beta_0$ represents the average demand in **_total bags_** when price is \$ $0$.



In [22]:
# since statsmodels expects variable names to not have spaces
# use a list comprehension to replace  spaces with underscores
# make sure you understand how/why this works:
avocado.columns = [c.replace(' ', ' ') for c in avocado.columns]

In [23]:
# take a look at the resulting dataframe

avocado.columns

Index(['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770',
       'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year',
       'region'],
      dtype='object')

In [24]:
# run the regression using statsmodels
# call the result object res
# for demand, use 'Total Bags'

formula = 'Total Bags - AveragePrice - C(region)'
res = smf.ols('formula, data ~ avocado').fit()

TypeError: ignored

In [ ]:
# print the regression summary table

print(res.summary())

**Edit this cell**
1. The R$^2$ is **___0.765___**, this means that **76.5**\% of the variation in demand is explained by the model.
1. The intercept ($\beta_0$) is **__1.847e+05___________** and is statistically **significant (choose one)**. This is the average demand in **_____Total bags_________** when price is \$0.
1. The coefficient for Boston is **____8.097e+04____________**, which means that the average demand in Boston is **____________** **fewer (pick one)** bags than in **___all other cities______________**. This difference is statistically **significant **
1. The coefficient for price is **__8121.8105_____________**, which means that when price increases by \$1 in any region, demand increases by on average **___8121.8105___________** bags. This effect is statistically **insignificant (pick one)** which means that there is enough evidence to show that price **does not (choose one)** have an **negative (pick one)** impact on demand.
1. The region with the highest demand is **___New York_______________**.

# 3. Putting what we've learned all together (the power of programming) (2 points - extra point for bonus, try your best - there may or may not be a prize involved)

In this exercise, you will run one regression for each region and each avocado type, i.e. one regression of total bags sold on price for Philly conventional, one for Philly organic, one for NY conventional, one for NY organic, and so on... (think loop). 

You will then visualize the coefficients for conventional and organic price by region on a scatter plot (each dot's (x, y) coordinates = one region's (price coefficient for conventional, price coefficient for organic). 

To do this break down the problem to the following components (and then put it all together):

* Suppose you have selected a region $r$ and avocado type $t$, how would you run the regression?
* How can you extract just the price coefficient given a regression result object `res`?
* For some region, `r`, and some avocado type `t`, how would you select only the rows pertaining to region `r` and type `t`?
* How can you loop through regions and types? (Need 2 nested "for" loops, one inside the other)
* Suppose you have named the conventional and organic price coefficients for region `r` as `bConv` and `bOrg`, how would you create a list that looks like: `[region, conventional coeff, organic coefficient]`?
* Given a list `results`, how would you *append* the list above to `results`? i.e. make keep adding to `results`: `[[region1, conventional coeff1, organic coeff1], [region2, conventional coeff2, organic coeff2], [region3, conventional coeff3, organic coeff3],...]`
* Once you've run every regression and have a complete `results` list, convert it to a dataframe using `df = pd.DataFrame(results)`, give it some column names.

The final code might have a structure similar to the pseudo code below:

```
results = list()
for r in ##all the unique regions##:
    for t in ##all the unique types##:
        temp = # Select the appropriate rows of df
        # run the regression using temp
        if ##this iteration represents conventional avocados##:
            ##extract the price parameter assign it to the variable bConv##
        else:
            ##extract the price parameter and name it bOrg##
    ##append the results with newest [region, conventional price coeff, organic price coefficient]##
df = pd.DataFrame(results) # convert to dataframe
df.columns = ['region', 'beta_conv', 'beta_org'] # rename dataframe columns
```

The resulting dataframe of coefficients should look something like:

| region | beta_conv | beta_org |
|-|-|-|
| Albany |8.579340e+02	 |-3168.491384|
| Atlanta| -3.377365e+05 | -6352.937420 |
| BaltimoreWashington | -3.251570e+05 | -15537.003211 |
|...|...|...|...|

(note numbers above are illustrative, not necessarily representative of actual results)


In [ ]:
# first re-read the original avocado dataframe to use for this problem
avocado = pd.read_csv(fpath + 'avocado.csv', index_col = 0)

In [ ]:
# rename columns to get rid of spaces


In [ ]:
# run all the regressions
# create a dataframe with region, beta_conv, beta_org as the 3 columns 
# using the regression results




In [ ]:
# Get rid of the TotalUS region from the regression results dataframe



In [ ]:
# show the full resulting table



In [ ]:
# make scatter plot of the coefficients
# change aesthetics so the dots are not a single blob




In [ ]:
# which regions have upward sloping demand curve for organic (as price increases, so did demand)?
# use code to show these rows of df



In [ ]:
# which regions have upward sloping demand curve for conventional (as price increases, so did demand)?
# use code to show these rows of df



Why might the relationship between price and demand be upward sloping instead of the expected downward sloping demand curve theorized in economics? i.e. the law of demand, when price increases, demand decreases. 

**Your answer here**